# A Closed Form Reduced Order Electrochemical Model for Lithium-Ion Cells

## Introduction

This model considers a Li-ion cell connected to an external load in a circuit. We consider that that a 3-dimensional cylindrical cell can be opened up as a sheet of paper of negligible thickness, now also assume that no change in any physical quantity occurs along the width. This turn our 3D system to a 1D one. 

Now according to standard conventions let's assume that our cell consists of **3 regions**, the **-ve electrode**, **the separator** and the **+ve electrode**. Also assume that the cell has collectors at end of either electrode. We'll be having the standard reactions in the +ve and -ve electrodes as below:

*-ve electrode:*

<div align=center>LiC₆ → x.Li⁺ + Li₁₋ₓC₆ + x.e⁻</div>

*+ve electrode:*

<div align=center>x.Li⁺ + LiMO₂ + x.e⁻ → Li₁₊ₓMO₂</div>


With this our system should look like below.

<div align=center><img src="../assets/senthil/li_cell.png"></div>

Assume that we have `x = 0` at the -ve electrode and `x = L` at the +ve electrode, L being the cell length. Length of the regions from left to right being lₙ, lₛ and lₚ respectively.

## The difference
This model build upon the SenthilModel where the reaction rate was assumed constant. However, that assumption is not accurate as cell reaction rate changes with time, to accomodate for the above condition, it is assusmed that the cell reaction rate follows an exponentially decreasing profile. This model is also called the Non uniform reduced order model, henceforth **NUROM**.


## Importing necessary libraries
To get started import the CaseStudy's senthil model (as sm for convinence) as that's the model which we're simulating. Also import Gadfly as that's the plotting library we're gonna use. To see the model constants [check here.](../src/ashwini/constants/constants.jl)

In [15]:
import LiionBatteryModels.AshwiniModel as am 
# using Gadfly

# set_default_plot_size(800px, 600px)

## Cell Voltage for constant current
In this example we'll be showing the cases of *constant current*. The example describes all the internal model variables also.

We start with a current of 1C or 13.5mA over a time range of 25000s with intervals of 10s.

In [16]:
I = 13.5
tspan = 10
t = Float64.(0:10:5000)

501-element Vector{Float64}:
    0.0
   10.0
   20.0
   30.0
   40.0
   50.0
   60.0
   70.0
   80.0
   90.0
    ⋮
 4920.0
 4930.0
 4940.0
 4950.0
 4960.0
 4970.0
 4980.0
 4990.0
 5000.0

We start with calculating the initial model parameters dₙ₁ and dₚ₁.

In [17]:
dₚ₁ = am.dₚ₁₀(I)
dₙ₁ = am.dₙ₁₀(I)

println("dₚ₁= ", dₚ₁,", dₙ₁ = ", dₙ₁)

dₚ₁= 0.060844230693563414, dₙ₁ = 0.06088358249081125


We calculate the secondary model parameters dₙ₂ and dₚ₂.

In [18]:
dₚ₂ = am.dₚ₂(dₚ₁)
dₙ₂ = am.dₙ₂(dₙ₁)

println("dₚ₂= ", dₚ₂,", dₙ₂ = ", dₙ₂)

dₚ₂= 1.9899107621234524, dₙ₂ = 1.9899042701176592


Let's create arrays to store the time series of internal variables that will be calculated along the way

In [19]:
# Stores for required time series of variables

# Reaction rate @x = 0 & x = lₙ, lₙ + lₛ & x = L
jₙ₀  = zeros(size(t)[1])
jₙₗ  = zeros(size(t)[1])

jₚ₀ = zeros(size(t)[1]);
jₚₗ = zeros(size(t)[1]);


# Average concentrations in different regions.
c₂̄ₙ = zeros(size(t)[1])
c₂̄ₚ = zeros(size(t)[1])
c₂̄ₛ = zeros(size(t)[1])

# Electrolyte concentration @x = 0 & x = lₙ, lₙ + lₛ & x = L
c₂ₙ₀ = zeros(size(t)[1])
c₂ₙₗ = zeros(size(t)[1]) # c₂ᵢₙ

c₂ₘ = zeros(size(t)[1])

c₂ₚ₀ = zeros(size(t)[1]) # c₂ᵢₚ
c₂ₚₗ = zeros(size(t)[1]) 

# Average electrolyte concentration in the anode, separator and cathode
c₂ₙₐ = zeros(size(t)[1])
c₂ₛₐ = zeros(size(t)[1])
c₂ₚₐ = zeros(size(t)[1])


# Solid concentration @x = 0, lₙ, lₙ + lₛ, L
c₁ₙ₀ = zeros(size(t)[1])
c₁ₙₗ = zeros(size(t)[1])

c₁ₚ₀ = zeros(size(t)[1])
c₁ₚₗ = zeros(size(t)[1])

# Radial gradient of Solid concentration @x = 0, lₙ, lₙ + lₛ, L
c₁ₙᵣ₀ = zeros(size(t)[1])
c₁ₙᵣₗ = zeros(size(t)[1])

c₁ₚᵣ₀ = zeros(size(t)[1])
c₁ₚᵣₗ = zeros(size(t)[1])

# Surface solid concentration @x = 0, lₙ, lₙ + lₛ, L
cₛₙ₀ = zeros(size(t)[1])
cₛₙₗ = zeros(size(t)[1])

cₛₚ₀ = zeros(size(t)[1])
cₛₚₗ = zeros(size(t)[1])

# Electrolyte potential @x= 0, lₙ, lₙ + lₛ, L
ϕ₂ₙ₀ = zeros(size(t)[1])
ϕ₂ₙₗ = zeros(size(t)[1]) # ϕ₂ᵢₙ

ϕ₂ₚ₀ = zeros(size(t)[1]) # ϕ₂ᵢₚ
ϕ₂ₚₗ = zeros(size(t)[1])


# Solid potential @x = 0, lₙ, lₙ + lₛ, L
ϕ₁ₙ₀ = zeros(size(t)[1])
ϕ₁ₙₗ = zeros(size(t)[1])

ϕ₁ₚ₀ = zeros(size(t)[1])
ϕ₁ₚₗ = zeros(size(t)[1])

# Voltage accross the cell terminals
V = zeros(size(t)[1]);

### Initial value storing.
Lets store all the initial values of the time series we mentioned above.

In [20]:
println("Initial values of the variables:\n")

# Reaction rate @x = 0 & x = lₙ, lₙ + lₛ & x = L

jₙ₀[1] = am.jₙ(0, dₙ₁, dₙ₂, I)
jₙₗ[1] = am.jₙ(am.lₙ, dₙ₁, dₙ₂, I)

jₚ₀[1] = am.jₚ(am.L - am.lₚ, dₚ₁, dₚ₂, I)
jₚₗ[1] = am.jₚ(am.L, dₚ₁, dₚ₂, I)

println("Reaction rates jₙ₀ = ", jₙ₀[1], ", jₚₗ = ", jₚₗ[1])

# Average electrolyte concentration in the anode, separator and cathode

c₂̄ₙ[1] = am.c₂₀
c₂̄ₛ[1] = am.c₂₀
c₂̄ₚ[1] = am.c₂₀

# Electrolyte concentration @x = 0 & x = lₙ, lₙ + lₛ & x = L

c₂ₙ₀[1] = am.c₂₀
c₂ₙₗ[1] = am.c₂₀

c₂ₘ[1] = am.c₂₀

c₂ₚ₀[1] = am.c₂₀
c₂ₚₗ[1] = am.c₂₀

# Electrolyte potential @x= 0, lₙ, lₙ + lₛ, L
ϕ₂ₙₗ[1] = am.ϕ₂ᵢₙ(I, c₂ₙₗ[1], c₂ₘ[1]) # ϕ₂ᵢₙ
ϕ₂ₙ₀[1] = am.ϕ₂ₙ(0, I, c₂ₙ₀[1], c₂ₘ[1], ϕ₂ₙₗ[1], dₙ₁)

ϕ₂ₚₗ[1] = am.ϕ₂ᵢₚ(I, c₂ₚ₀[1], c₂ₘ[1])
ϕ₂ₚ₀[1] =  am.ϕ₂ₚ(am.L, I, c₂ₚₗ[1], c₂ₘ[1], ϕ₂ₚₗ[1], dₚ₁)  # ϕ₂ᵢₚ

println("Electrolyte potentials ϕ₂ₙ₀ = ", ϕ₂ₙ₀[1], ", ϕ₂ₚₗ = ", ϕ₂ₚₗ[1])

# Solid concentration @x = 0, lₙ, lₙ + lₛ, L

c₁ₙ₀[1] = am.c₁ₙ₀
c₁ₙₗ[1] = am.c₁ₙ₀

c₁ₚ₀[1] = am.c₁ₚ₀
c₁ₚₗ[1] = am.c₁ₚ₀

println("Solid concentrations c₁ₙ₀ = ", c₁ₙ₀[1], ", c₁ₚₗ = ", c₁ₚₗ[1])

# Radial gradient of Solid concentration @x = 0, lₙ, lₙ + lₛ, L
c₁ₙᵣ₀[1] = 0
c₁ₙᵣₗ[1] = 0

c₁ₚᵣ₀[1] = 0
c₁ₚᵣₗ[1] = 0

# Surface solid concentration @x = 0, lₙ, lₙ + lₛ, L
cₛₙ₀[1] = am.cₛₙ(jₙ₀[1], c₁ₙ₀[1], c₁ₙᵣ₀[1])
cₛₙₗ[1] = am.cₛₙ(jₙₗ[1], c₁ₙₗ[1], c₁ₙᵣₗ[1])

cₛₚ₀[1] = am.cₛₚ(jₚ₀[1], c₁ₚ₀[1], c₁ₚᵣ₀[1])
cₛₚₗ[1] = am.cₛₚ(jₚₗ[1], c₁ₚₗ[1], c₁ₚᵣₗ[1])

println("Surface solid concentrations cₛₙ₀ = ", cₛₙ₀[1], ", cₛₚₗ = ", cₛₚₗ[1])

# Solid potential @x = 0, lₙ, lₙ + lₛ, L
ϕ₁ₙ₀[1] = am.ϕ₁ₙ(jₙ₀[1], cₛₙ₀[1], c₂ₙ₀[1], ϕ₂ₙ₀[1])
ϕ₁ₙₗ[1] = am.ϕ₁ₙ(jₙₗ[1], cₛₙₗ[1], c₂ₙₗ[1], ϕ₂ₙₗ[1])

ϕ₁ₚ₀[1] = am.ϕ₁ₚ(jₚ₀[1], cₛₚ₀[1], c₂ₚ₀[1], ϕ₂ₚ₀[1])
ϕ₁ₚₗ[1] = am.ϕ₁ₚ(jₚₗ[1], cₛₚₗ[1], c₂ₚₗ[1], ϕ₂ₚₗ[1])

println("Solid potentials ϕ₁ₙ₀ = ", ϕ₁ₙ₀[1], ", ϕ₁ₚₗ = ", ϕ₁ₚₗ[1])

# Calculating the voltage
V[1] = ϕ₁ₚₗ[1] - ϕ₁ₙ₀[1]

println("Voltage V = ", V[1]);

# Updating model parameters
dₙ₁ = am.dₙ₁(I, cₛₙ₀[1], cₛₙₗ[1], c₂ₙₗ[1], c₂ₙ₀[1], c₂̄ₙ[1])
dₚ₁ = am.dₚ₁(I, cₛₚ₀[1], cₛₚₗ[1], c₂ₚₗ[1], c₂ₚ₀[1], c₂̄ₚ[1])

dₚ₂ = am.dₚ₂(dₚ₁);
dₙ₂ = am.dₙ₂(dₙ₁);

Initial values of the variables:

Reaction rates jₙ₀ = 9.423553405093455e-6, jₚₗ = -5.231137697239635e-6
Electrolyte potentials ϕ₂ₙ₀ = 0.004873135697747667, ϕ₂ₚₗ = -0.001755472970989933
Solid concentrations c₁ₙ₀ = 19528.6, c₁ₚₗ = 8000.999999999999
Surface solid concentrations cₛₙ₀ = 19442.303723396577, cₛₚₗ = 8017.91636691137
Solid potentials ϕ₁ₙ₀ = 0.3658711499991764, ϕ₁ₚₗ = 4.095046161514364
Voltage V = 3.729175011515187


### Model solving
We start with calculation of solid concentration field and then calculate the electrolyte concentration field. From this we calculate electrolyte potential field. Now from this we calculate the solid potential field, from which we can Finally calculate the cell voltage.

Note that all of this iteratively moving from t=0 to t=25000s in time deltas of 10s.

In [33]:
for i = 2:20
    jₙ₀[i] = am.jₙ(0, dₙ₁, dₙ₂, I)
    jₙₗ[i] = am.jₙ(am.lₙ, dₙ₁, dₙ₂, I)

    jₚ₀[i] = am.jₚ(am.L - am.lₚ, dₚ₁, dₚ₂, I)
    jₚₗ[i] = am.jₚ(am.L, dₚ₁, dₚ₂, I)

    # Calculation of solid concentration field.
    c₁ₙ₀[i] = am.c₁ₙ(tspan, jₙ₀[i], c₁ₙ₀[i-1])
    c₁ₙₗ[i] = am.c₁ₙ(tspan, jₙₗ[i], c₁ₙₗ[i-1])

    c₁ₚ₀[i] = am.c₁ₚ(tspan, jₚ₀[i], c₁ₚ₀[i-1])
    c₁ₚₗ[i] = am.c₁ₚ(tspan, jₚₗ[i], c₁ₚₗ[i-1])

    # Calculation of Radial gradient of Solid concentration field.
    c₁ₙᵣ₀[i] = am.c₁ₙᵣ(tspan, jₙ₀[i], c₁ₙᵣ₀[i-1])
    c₁ₙᵣₗ[i] = am.c₁ₙᵣ(tspan, jₙₗ[i], c₁ₙᵣₗ[i-1])

    c₁ₚᵣ₀[i] = am.c₁ₚᵣ(tspan, jₚ₀[i], c₁ₚᵣ₀[i-1])
    c₁ₚᵣₗ[i] = am.c₁ₚᵣ(tspan, jₚₗ[i], c₁ₚᵣₗ[i-1])

    # Calculation of solid surface concentration field.
    cₛₙ₀[i] = am.cₛₙ(jₙ₀[i], c₁ₙ₀[i], c₁ₙᵣ₀[i])
    cₛₙₗ[i] = am.cₛₙ(jₙₗ[i], c₁ₙₗ[i], c₁ₙᵣₗ[i])

    cₛₚ₀[i] = am.cₛₚ(jₚ₀[i], c₁ₚ₀[i], c₁ₚᵣ₀[i])
    cₛₚₗ[i] = am.cₛₚ(jₚₗ[i], c₁ₚₗ[i], c₁ₚᵣₗ[i])

    # Calculation of electrolyte concentration field
    params = am.param_solver(
        I, tspan, 
        c₂̄ₙ[i-1], c₂̄ₚ[i-1], c₂̄ₛ[i-1], 
        c₂ₙ₀[i-1], c₂ₚₗ[i-1], 
        jₙ₀[i], jₚₗ[i]
    )

    n0 = params[1]; n1 = params[2]; n2 = params[3]; n3 = params[4];
    p0 = params[5]; p1 = params[6]; p2 = params[7]; p3 = params[8];
    s0 = params[9]; s1 = params[10]; s2 = params[11];

    c₂̄ₙ[i] = params[12];
    c₂̄ₚ[i] = params[13];
    c₂̄ₛ[i] = params[14];
    c₂ₙ₀[i] = params[15];
    c₂ₚₗ[i] = params[16];

    paramsₙ = [n0, n2, n3];
    paramsₛ = [s0, s1, s2];
    paramsₚ = [p0, p2, p3];

    c₂ₙₗ[i] = am.c₂(am.lₙ, paramsₙ);
    c₂ₚ₀[i] = am.c₂(am.lₙ + am.lₛ, paramsₛ);

    ϕ₂ₙ₀[i] = am.ϕ₂(0, I, paramsₙ, paramsₛ, dₙ₁)
    ϕ₂ₙₗ[i] = am.ϕ₂(am.lₙ, I, paramsₙ, paramsₛ, dₙ₁)
    ϕ₂ₚ₀[i] = am.ϕ₂(am.L - am.lₚ, I, paramsₚ, paramsₛ, dₙ₁)
    ϕ₂ₚₗ[i] = am.ϕ₂(am.L, I, paramsₚ, paramsₛ, dₙ₁)

    ϕ₁ₙ₀[i] = am.ϕ₁(0, I, cₛₙ₀[i], paramsₙ,paramsₛ, dₙ₁)
    ϕ₁ₙₗ[i] = am.ϕ₁(am.lₙ, I, cₛₙₗ[i], paramsₙ, paramsₛ, dₙ₁)
    ϕ₁ₚ₀[i] = am.ϕ₁(am.L - am.lₚ, I, cₛₚ₀[i], paramsₚ, paramsₛ, dₚ₁)
    ϕ₁ₚₗ[i] = am.ϕ₁(am.L, I, cₛₚₗ[i], paramsₚ, paramsₛ, dₚ₁)

    V[i] = ϕ₁ₚₗ[i] - ϕ₁ₙ₀[i]
    
    dₙ₁ = am.dₙ₁(I, cₛₙ₀[i], cₛₙₗ[i], paramsₙ)
    dₚ₁ = am.dₙ₁(I,  cₛₚ₀[i], cₛₚₗ[i], paramsₙ)

    dₚ₂ = am.dₚ₂(dₚ₁)
    dₙ₂ = am.dₙ₂(dₙ₁)
end


Reaction rates jₙ₀ = 9.5818394588947e-6, jₚₗ = -5.374029132305806e-6
Solid concentrations c₁ₙ₀ = 19505.60358529865, c₁ₚₗ = 8019.967161643432
Radial gradients c₁ₙᵣ₀ = -1.379784882080837e7, c₁ₚᵣₗ = 1.6735730861851992e7
Surface concentrations cₛₙ₀ = 19378.43537750573, cₛₚₗ = 8069.860742769513
D₂ₙ: 1.6467514430027962e-9
D₂ₛ: 7.38638910814326e-10
D₂ₚ: 1.6617445703334925e-9

c₂̄ₙ = 1203.8330768297078, c₂̄ₚ = 1196.7867889036788, c₂̄ₛ = 1200.5079055642184

[1202.065291065957; 0.0; 1.7526764594712886e8; 6.575336624901371e10; 1198.9449503111325; 0.0; -1.3546491299810377e8; -4.9851935795386765e10; 1202.065291065957; -104568.33362895326; 1.8565837826565277e7; 1203.8330768297078; 1196.7867889036788; 1200.5079055642184; 1204.7027669844738; 1195.7287394853658;;]
Electrolyte potentials ϕ₂ₙ₀ = 0.005006979362328769, ϕ₂ₚₗ = -0.003968920350848643
Solid potentials ϕ₁ₙ₀ = 0.36754378316322045, ϕ₁ₚₗ = 4.092236231787132
Voltage V = 3.724692448623912
Reaction rates jₙ₀ = 9.42758369609562e-6, jₚₗ = -5.2317580189

DomainError: DomainError with -0.1722389191494782:
Exponentiation yielding a complex result requires a complex argument.
Replace x^y with (x+0im)^y, Complex(x)^y, or similar.

In [23]:
# plot(
#     layer(x=t[1:17], y=jₙ₀[1:17], Geom.line, color=[colorant"orange"]),
#     layer(x=t[1:17], y=jₙₗ[1:17], Geom.line, color=[colorant"lightgreen"]),
#     layer(x=t[1:17], y=jₚ₀[1:17], Geom.line, color=[colorant"blue"]),
#     layer(x=t[1:17], y=jₚₗ[1:17], Geom.line, color=[colorant"red"]),
#     # layer(x=t, y=V, Geom.line,color=[colorant"green"]),
#     # layer(x=t, y=c₂ₗₜ, Geom.line,color=[colorant"lightblue"]),
#     Guide.title("Variation of reaction rate with time"),
#     Guide.xlabel("Time (in s)"),
#     Guide.ylabel("jₖ (in mol/m³)"),
# )

In [24]:
# plot(
#     layer(x=t[1:17], y=c₂ₙ₀[1:17], Geom.line, color=[colorant"orange"]),
#     layer(x=t[1:17], y=c₂ₙₗ[1:17], Geom.line, color=[colorant"lightgreen"]),
#     layer(x=t[1:17], y=c₂ₚ₀[1:17], Geom.line, color=[colorant"blue"]),
#     layer(x=t[1:17], y=c₂ₚₗ[1:17], Geom.line, color=[colorant"red"]),
#     Guide.title("Variation of electrolyte concentration with time"),
#     Guide.xlabel("Time (in s)"),
#     Guide.ylabel("Cₛₖ (in mol/m³)"),
# )

In [25]:
# plot(
#     layer(x=t[1:17], y=c₁ₙ₀[1:17], Geom.line, color=[colorant"orange"]),
#     layer(x=t[1:17], y=c₁ₙₗ[1:17], Geom.line, color=[colorant"lightgreen"]),
#     layer(x=t[1:17], y=c₁ₚ₀[1:17], Geom.line, color=[colorant"blue"]),
#     layer(x=t[1:17], y=c₁ₚₗ[1:17], Geom.line, color=[colorant"red"]),
#     Guide.title("Variation of average solid concentration with time"),
#     Guide.xlabel("Time (in s)"),
#     Guide.ylabel("C₁ₖ (in mol/m³)"),
# )

In [26]:
# plot(
#     layer(x=t[1:17], y=cₛₙ₀[1:17], Geom.line, color=[colorant"orange"]),
#     layer(x=t[1:17], y=cₛₙₗ[1:17], Geom.line, color=[colorant"lightgreen"]),
#     layer(x=t[1:17], y=cₛₚ₀[1:17], Geom.line, color=[colorant"blue"]),
#     layer(x=t[1:17], y=cₛₚₗ[1:17], Geom.line, color=[colorant"red"]),
#     Guide.title("Variation of Suface solid concentration with time"),
#     Guide.xlabel("Time (in s)"),
#     Guide.ylabel("Cₛₖ (in mol/m³)"),
# )

In [27]:

# plot(
#     layer(x=t[1:17], y=V[1:17], Geom.line, color=[colorant"orange"]),
#     Guide.title("Variation of Voltage"),
#     Guide.xlabel("Time (in s)"),
#     Guide.ylabel("Voltage (in V)"),
# )
